In [1]:
import pandas as pd
import numpy as np

# 1) Carreguem els datasets petits
sales = pd.read_csv("videogamesales.csv")
meta  = pd.read_csv("metacritic_games_data.csv")

print("sales:", sales.shape)
print("meta :", meta.shape)

display(sales.head())
display(meta.head())


# 2) Steam: carrega una mostra (segur) només amb columnes necessàries
steam = pd.read_csv(
    "steam_reviews.csv",
    usecols=["app_name", "review_score", "review_votes"],
    nrows=300_000
)

display(steam.head())

# Neteja bàsica + tipus
steam = steam.dropna(subset=["app_name", "review_score"])
steam["review_score"] = pd.to_numeric(steam["review_score"], errors="coerce")
steam["review_votes"] = pd.to_numeric(steam["review_votes"], errors="coerce").fillna(0)
steam = steam.dropna(subset=["review_score"])


# 3) Agregació per joc (1 fila = 1 joc)
# - review_count: nº reviews
# - avg_review_score: mitjana de review_score
# - controversy_score: variabilitat (std) del review_score
# - votes_sum: suma de vots
steam_agg = (
    steam
    .groupby("app_name")
    .agg(
        review_count=("review_score", "count"),
        avg_review_score=("review_score", "mean"),
        controversy_score=("review_score", "std"),
        votes_sum=("review_votes", "sum")
    )
    .reset_index()
)

# std pot sortir NaN si un joc té 1 sola review a la mostra
steam_agg["controversy_score"] = steam_agg["controversy_score"].fillna(0)

# Engagement proxy: reviews + vots
steam_agg["engagement_proxy"] = steam_agg["review_count"] * (1 + np.log1p(steam_agg["votes_sum"]))

display(steam_agg.head())


# 4) Guardar per unir després amb sales + meta
steam_agg.to_csv("steam_aggregated.csv", index=False)
print("Creat steam_aggregated.csv amb files:", len(steam_agg))


sales: (16598, 11)
meta : (17944, 10)


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


,name,platform,r-date,score,user score,developer,genre,players,critics,users
0,The Legend of Zelda: Ocarina of Time,Nintendo64,"November 23, 1998",99,9.1,Nintendo,"Action Adventure,Fantasy",1 Player,22,5749
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",98,7.4,NeversoftEntertainment,"Sports,Alternative,Skateboarding",1-2,19,647
2,Grand Theft Auto IV,PlayStation3,"April 29, 2008",98,7.6,RockstarNorth,"Action Adventure,Modern,Modern,Open-World",1 Player,64,3806
3,SoulCalibur,Dreamcast,"September 8, 1999",98,8.5,Namco,"Action,Fighting,3D",1-2,24,324
4,Grand Theft Auto IV,Xbox360,"April 29, 2008",98,7.9,RockstarNorth,"Action Adventure,Modern,Modern,Open-World",1 Player,86,3364


,app_name,review_score,review_votes
0,Counter-Strike,1,0
1,Counter-Strike,1,1
2,Counter-Strike,1,0
3,Counter-Strike,1,0
4,Counter-Strike,1,1


,app_name,review_count,avg_review_score,controversy_score,votes_sum,engagement_proxy
0,3D Ultra Minigolf Adventures Deluxe,37,0.297297,0.967955,18,145.944242
1,A New Beginning - Final Cut,446,0.242152,0.971328,29,1962.934032
2,A.V.A. Alliance of Valiant Arms™,5074,-0.057942,0.998418,785,38902.138765
3,APB Reloaded,12604,0.188988,0.982018,1411,104017.817517
4,Aces of the Galaxy,19,-0.052632,1.025978,4,49.579320


Creat steam_aggregated.csv amb files: 117


In [2]:
# =========================================================
# PART 2:
# - Normalitzar noms
# - Unir Sales + Meta + SteamAgg
# - Arreglar escalat (critics 0-100 -> 0-10)
# - Export final per Flourish
# =========================================================

import pandas as pd
import numpy as np
import re

# -------------------------
# 1) Carrega datasets base
# -------------------------
sales = pd.read_csv("videogamesales.csv")
meta  = pd.read_csv("metacritic_games_data.csv")
steam_agg = pd.read_csv("steam_aggregated.csv")

print("sales:", sales.shape)
print("meta :", meta.shape)
print("steam_agg:", steam_agg.shape)

# -------------------------
# 2) Funció per normalitzar noms
# -------------------------
def normalize_name(s) -> str:
    if pd.isna(s):
        return ""
    s = str(s).lower()
    s = re.sub(r"\(.*?\)", "", s)         # elimina ( ... )
    s = re.sub(r"[^a-z0-9 ]", "", s)      # elimina símbols
    s = re.sub(r"\s+", " ", s).strip()    # espais múltiples
    return s

# -------------------------
# 3) Crea claus de join 
# -------------------------
sales["name_norm"] = sales["Name"].apply(normalize_name)
meta["name_norm"]  = meta["name"].apply(normalize_name)

steam_name_col = "app_name" if "app_name" in steam_agg.columns else "name"
steam_agg["name_norm"] = steam_agg[steam_name_col].apply(normalize_name)

# -------------------------
# 4) Unió Sales + Meta 
# -------------------------
sales_meta = pd.merge(
    sales,
    meta,
    on="name_norm",
    how="inner",
    suffixes=("_sales", "_meta")
)
print("sales_meta:", sales_meta.shape)

# -------------------------
# 5) Afegir Steam 
# -------------------------
final_df = pd.merge(
    sales_meta,
    steam_agg,
    on="name_norm",
    how="left"
)
print("final_df:", final_df.shape)

# -------------------------
# 6) Conversions + variables derivades coherents
# -------------------------
# Metacritic: score (0-100), user score (0-10)
final_df["score"] = pd.to_numeric(final_df["score"], errors="coerce")
final_df["user score"] = pd.to_numeric(final_df["user score"], errors="coerce")

# Critic en escala 0-10
final_df["critic_score_10"] = final_df["score"] / 10

# Gap coherent en escala 0-10
final_df["metacritic_gap_10"] = final_df["critic_score_10"] - final_df["user score"]



# Marca si hi ha dades Steam 
if "review_count" in final_df.columns:
    final_df["has_steam_data"] = final_df["review_count"].notna()
else:
    final_df["has_steam_data"] = False

# -------------------------
# 7) Selecció de columnes finals 
# -------------------------
cols_out = [
    # Identificadors / dimensions
    "Name", "Platform", "Year", "Genre", "Publisher",
    # Vendes
    "Global_Sales",
    # Metacritic (original + harmonitzat)
    "score", "critic_score_10", "user score", "players", "critics", "users",
    "metacritic_gap_10",
    # Steam (si existeixen)
    "review_count", "votes_sum", "avg_review_score", "controversy_score", "engagement_proxy",
    "has_steam_data"
]

cols_out = [c for c in cols_out if c in final_df.columns]

final_out = final_df[cols_out].copy()

print("final_out:", final_out.shape)
display(final_out.head())

# -------------------------
# 8) Export final per Flourish
# -------------------------
final_out.to_csv("final_dataset_for_flourish.csv", index=False)
print("Creat: final_dataset_for_flourish.csv")


sales: (16598, 11)
meta : (17944, 10)
steam_agg: (117, 6)
sales_meta: (21003, 22)
final_df: (21003, 28)
final_out: (21003, 19)


,Name,Platform,Year,Genre,Publisher,Global_Sales,score,critic_score_10,user score,players,critics,users,metacritic_gap_10,review_count,votes_sum,avg_review_score,controversy_score,engagement_proxy,has_steam_data
0,Wii Sports,Wii,2006.0,Sports,Nintendo,82.74,76,7.6,8.0,No Online Multiplayer,51,429,-0.4,NaN,NaN,NaN,NaN,NaN,False
1,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,35.82,82,8.2,8.4,Up to 12,73,982,-0.2,NaN,NaN,NaN,NaN,NaN,False
2,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,33.00,80,8.0,8.1,1-4,73,266,-0.1,NaN,NaN,NaN,NaN,NaN,False
3,Tetris,GB,1989.0,Puzzle,Nintendo,30.26,85,8.5,7.7,No info,7,24,0.8,NaN,NaN,NaN,NaN,NaN,False
4,Tetris,GB,1989.0,Puzzle,Nintendo,30.26,80,8.0,7.3,4 Online,11,16,0.7,NaN,NaN,NaN,NaN,NaN,False


Creat: final_dataset_for_flourish.csv


In [3]:
final_out.to_csv(r"C:\Users\Roger\Desktop\final_dataset_for_flourish.csv", index=False)
